In [ ]:
# Imports
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
from enum import Enum
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.multiprocessing as mp
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
import seaborn as sns
import mlflow
import time

%load_ext autoreload
%autoreload 2

from forecaster.scalers import *
from forecaster.preprocessing import TimeseriesDataSet, Granularity
from forecaster.models import *
from forecaster.training import EarlyStopper, ModelTrainer, TimeseriesForecaster
from forecaster.evaluation import evaluate_series, top_n_by_metric, get_full_results_dict, align_ground_truth

In [ ]:
# Environment setup
os.makedirs(f"../logs/mlruns/.trash", exist_ok=True)
torch.cuda.empty_cache()
mp.set_start_method('spawn', force=True)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")

In [ ]:
# Data set configuration
PATH = "../data/train_data.csv" 
GRANULARITY = Granularity.HOURLY
N_SERIES = 137 # Number of parallel time series in the dataset

# Model Data configuration
SCALER = LogStandardScaler
TRAIN_VALIDATION_SPLIT = 0.8
INPUT_SIZE = 48
OUTPUT_SIZE = 24

# Feature toggles
USE_TIME_COVARIATES = True  # Include cyclical time covariates in the model inputs

# Training configuration
LR = 0.001
BATCH_SIZE = 512
N_EPOCHS = 100

# Model configuration
HIDDEN_DIM = 64
N_LAYERS = 4
DROPOUT = 0.3
# Logging configuration
USE_MLFLOW = True
SAVE_MODEL = False

In [ ]:
# Data Loading
pandas_df = pd.read_csv(PATH)
pandas_df.info()

In [ ]:
# Limit series for faster experimentation
pandas_df = pandas_df.iloc[:, :N_SERIES + 1]

In [ ]:
fig = go.Figure()
for i in range(min(N_SERIES, 10)):  # Plot only first 10 series
    fig.add_trace(go.Scatter(
        x=pandas_df["deviceTimestamp"],
        y=pandas_df.iloc[:, i + 1],
        mode="lines",
        name=f"Value_{i+1}"
    ))
fig.update_layout(
    title="All Series over Time",
    width=1200,
    height=400,
    xaxis_title="deviceTimestamp",
    yaxis_title="Value"
)
fig.show()

In [ ]:
dataset = TimeseriesDataSet(pandas_df, GRANULARITY, USE_TIME_COVARIATES, "deviceTimestamp", TRAIN_VALIDATION_SPLIT, SCALER, INPUT_SIZE, OUTPUT_SIZE)
N_COV = dataset.get_time_feature_count()

In [ ]:
series = pandas_df.columns[1]  # Select the first series for visualization
train_data = dataset.split_scaled_dict[series]["train"].flatten()
val_data = dataset.split_scaled_dict[series]["validation"].flatten()

train_idx = np.arange(len(train_data))
val_idx = np.arange(len(train_data), len(train_data) + len(val_data))

df_plot = pd.DataFrame({
    "Index": np.concatenate([train_idx, val_idx]),
    "Scaled Value": np.concatenate([train_data, val_data]),
    "Split": ["Train"] * len(train_data) + ["Validation"] * len(val_data)
})

fig = px.line(
    df_plot,
    x="Index",
    y="Scaled Value",
    color="Split",
    title=f"Scaled Data for {series} (Train/Validation Split)",
    width=1200,
    height=400
)
fig.add_vline(x=len(train_data)-1, line_dash="dash", line_color="red", annotation_text="Train/Validation Split")
fig.show()

In [ ]:

# data = dataset.split_scaled_dict["value_5"]["train"]
# print(f"value_5: mean={data.mean():.3f}, std={data.std():.3f}, "
#         f"min={data.min():.3f}, max={data.max():.3f}")
# data = dataset.split_scaled_dict["value_5"]["validation"]
# print(f"value_5: mean={data.mean():.3f}, std={data.std():.3f}, "
#         f"min={data.min():.3f}, max={data.max():.3f}")
# data = dataset.split_scaled_dict["value_133"]["train"]
# print(f"value_133: mean={data.mean():.3f}, std={data.std():.3f}, "
#         f"min={data.min():.3f}, max={data.max():.3f}")
# data = dataset.split_scaled_dict["value_133"]["validation"]
# print(f"value_133: mean={data.mean():.3f}, std={data.std():.3f}, "
#         f"min={data.min():.3f}, max={data.max():.3f}")

In [ ]:
corr = dataset.get_correlation_matrix()
plt.figure(figsize=(5, 4))
sns.heatmap(corr, cmap="coolwarm")
plt.title("Correlation Matrix of Time Series Values (Sorted)")
plt.show()

In [ ]:
# input_dim becomes 1 + N_COV when covariates are used
ModelClass = GRU
model_params = {
    "input_dim": 1 + N_COV,
    "hidden_dim": HIDDEN_DIM,
    "output_dim": OUTPUT_SIZE,
    "num_layers": N_LAYERS,
    "dropout": DROPOUT
}
model = ModelClass(**model_params)
with open("../logs/model_architecture.txt", "w") as f:
    f.write(str(model))

In [ ]:
# Model training
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-5)

early_stopper = EarlyStopper(patience=15, min_delta=1e-4) # Early stop on thresholded validation loss improvement
loss_fn = nn.L1Loss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=N_EPOCHS)
train_loader = dataset.get_train_dataloader(BATCH_SIZE, shuffle=True)
val_loader = dataset.get_validation_dataloader(BATCH_SIZE)

model_trainer = ModelTrainer(model, DEVICE, optimizer, loss_fn, scheduler, 
                             train_loader, val_loader,
                             early_stopper)

def print_metrics(epoch, metrics):
    print(f"Epoch {epoch}: Train RMSE {metrics['train_rmse']:.4f} | Train MAE {metrics['train_mae']:.4f} \
| Val RMSE {metrics['val_rmse']:.4f} | Val MAE {metrics['val_mae']:.4f} | \
LR {metrics['learning_rate']:.6f} | Epoch time {metrics['epoch_time']:.2f}s")

In [ ]:

history = model_trainer.fit(N_EPOCHS, on_epoch_end=print_metrics)
model = model_trainer.get_model()

total_training_time = np.sum(history["epoch_time"])
epoch_times = history["epoch_time"]
print(f"Total training time: {total_training_time:.2f}s ({total_training_time/60:.2f}m). Avg epoch time: {np.mean(epoch_times):.2f}s")

torch.save(model.state_dict(), "../logs/model.pt")

In [ ]:
try:
    model.eval()
except:
    model = ModelClass(**model_params)
    model.load_state_dict(torch.load("../logs/model.pt", map_location=DEVICE))
    model = model.to(DEVICE)
    model.eval()

In [ ]:
# Attention heatmap over validation set: rows=lag, cols=windows, colors=attention weights
model.eval()
key = pandas_df.columns[1]

if isinstance(model, (GRUAttention, LSTMAttention)):
    with torch.no_grad():
        val_dataloader = dataset.get_single_series_dataloader(key, "validation", BATCH_SIZE, shuffle=False)
        seq_len = dataset.input_size
        A_list = []
        for X_batch, y_batch in val_dataloader:
            inputs = X_batch.to(DEVICE)
            _, attn_weights = model(inputs, return_attention=True)
            A_list.append(attn_weights.cpu().numpy())
        A = np.vstack(A_list)  # shape (n_windows, seq_len)
        # timestamps for validation range
        val_timestamps = dataset.get_resampled_data()[dataset.timestamp_column].iloc[dataset.n_train:].reset_index(drop=True)
        # map attention windows into heatmap matrix M (rows = window index, cols = validation timestamps)
        n_windows = 0 if A.size == 0 else A.shape[0]
        M = np.full((n_windows, len(val_timestamps)), np.nan)
        for w in range(n_windows):
            end_col = min(w + seq_len, len(val_timestamps))
            M[w, w:end_col] = A[w, : end_col - w]

    # Limit to last week of validation timestamps
    if GRANULARITY == Granularity.HOURLY:
        last_week_mask = val_timestamps >= (val_timestamps.max() - pd.Timedelta(days=7))
    elif GRANULARITY == Granularity.DAILY:
        last_week_mask = val_timestamps >= (val_timestamps.max() - pd.Timedelta(days=7))
    elif GRANULARITY == Granularity.MONTHLY:
        last_week_mask = val_timestamps >= (val_timestamps.max() - pd.DateOffset(months=1))
    else:
        last_week_mask = np.ones(len(val_timestamps), dtype=bool)
    last_week_mask = last_week_mask.values.astype(bool)
    val_timestamps = val_timestamps[last_week_mask].reset_index(drop=True)
    M = M[:, last_week_mask]
    valid_row_mask = ~np.all(np.isnan(M), axis=1)
    M = M[valid_row_mask, :]

    # get values of series for the last week
    series_values = dataset.get_scaled_data(key)["validation"]
    series_values = series_values[last_week_mask].flatten()

In [ ]:
if isinstance(model, (GRUAttention, LSTMAttention)):
    fig_heat = go.Figure()

    # Heatmap
    fig_heat.add_trace(go.Heatmap(
        z=M,
        colorscale='Viridis',
        colorbar=dict(title='Attention'),
        x=val_timestamps,
        y=list(range(M.shape[0])),
        name='attention'
    ))

    # Series line on a secondary y-axis so it uses its own value scale
    fig_heat.add_trace(go.Scatter(
        x=val_timestamps,
        y=series_values,
        mode='lines',
        name=f'{key} values',
        line=dict(color='black', width=2),
        yaxis='y2'
    ))

    # Layout: add yaxis2 that overlays the heatmap y-axis
    fig_heat.update_layout(
        title='Attention heatmap for value_35 (timestamps on x-axis, rows=rolling windows)',
        xaxis=dict(title='Timestamp (validation range)'),
        yaxis=dict(title='Validation window index (rolling)'),
        yaxis2=dict(
            title=f'Scaled values for {key}',
            overlaying='y',
            side='right'
        ),
        width=1200,
        height=600,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
    )

    fig_heat.show()
    fig_heat.write_html("../logs/attention_heatmap.html", include_plotlyjs='cdn')

In [ ]:
# Train-Validation RMSE Plot (linear + log scale)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(history["train_rmse"], marker='o', label='Train RMSE')
axes[0].plot(history["val_rmse"], marker='x', label='Val RMSE')
axes[0].set_title('RMSE over Epochs (linear)')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('RMSE')
axes[0].legend()
axes[0].grid(True, which='both', alpha=0.3)

axes[1].plot(history["train_rmse"], marker='o', label='Train RMSE')
axes[1].plot(history["val_rmse"], marker='x', label='Val RMSE')
axes[1].set_yscale('log')
axes[1].set_title('RMSE over Epochs (log scale)')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('RMSE (log)')
axes[1].legend()
axes[1].grid(True, which='both', alpha=0.3)

plt.tight_layout()
plt.savefig("../logs/rmse.png")

In [ ]:
fig = plt.figure(figsize=(10, 5))
plt.plot(epoch_times, marker='o')
plt.axhline(y=np.mean(epoch_times), color='r', linestyle='--', label='Avg Epoch Time')
plt.legend()
plt.title('Epoch Times')
plt.xlabel('Epoch')
plt.ylabel('Time (s)')

plt.tight_layout()
fig.savefig("../logs/epoch_times.png", bbox_inches='tight', dpi=150)

In [ ]:
forecaster = TimeseriesForecaster(model, DEVICE, INPUT_SIZE, OUTPUT_SIZE)

print("Generating predictions for all series...")
start_time = time.time()
predictions_dict = forecaster.predict_all_series(dataset, BATCH_SIZE)
print(f"Predictions generated in {time.time() - start_time:.2f}s")

summary = evaluate_series(
    dataset=dataset,
    preds=predictions_dict,
    input_size=INPUT_SIZE,
    output_size=OUTPUT_SIZE
)

top_5 = top_n_by_metric(summary, n=5, metric='val_mae', reverse=False)
bottom_5 = top_n_by_metric(summary, n=5, metric='val_mae', reverse=True)

In [ ]:
def get_naive_predictions_dict(dataset, input_size, output_size):
    naive_preds = {}
    for key in dataset.split_unscaled_dict:
        naive_preds[key] = {}
        for split in ['train', 'validation']:
            data = dataset.get_unscaled_data(key)[split]
            preds = []
            # Match LSTM stride logic
            valid_split_points = range(input_size, len(data) + 1, output_size)
            for i in valid_split_points:
                if i - output_size >= 0:
                    pred_window = data[i - output_size : i]
                    if len(pred_window) == output_size:
                        preds.append(pred_window)
            
            if preds:
                naive_preds[key][split] = np.concatenate(preds).flatten()
            else:
                naive_preds[key][split] = np.array([])
    return naive_preds

naive_predictions_dict = get_naive_predictions_dict(dataset, INPUT_SIZE, OUTPUT_SIZE)

naive_summary = evaluate_series(
    dataset=dataset,
    preds=naive_predictions_dict,
    input_size=INPUT_SIZE,
    output_size=OUTPUT_SIZE
)

In [ ]:

print("Top 5 series by val_mae:")
for k in top_5:
    val = summary[k].get('val_mae', np.nan)
    print(k, f"{float(val):.2f}")
print("Bottom 5 series by val_mae:")
for k in bottom_5:
    val = summary[k].get('val_mae', np.nan)
    print(k, f"{float(val):.2f}")

full_results_dict = get_full_results_dict(dataset, predictions_dict, INPUT_SIZE, OUTPUT_SIZE)

# Plot MAE for all series (validation set), sorted by MAE
mae_vals = [float(summary[k]['val_mae']) for k in summary]
series_names = list(summary.keys())

# Sort by MAE
sorted_indices = np.argsort(mae_vals)
sorted_mae = [mae_vals[i] for i in sorted_indices]
sorted_series = [series_names[i] for i in sorted_indices]

fig = px.bar(
    x=sorted_series,
    y=sorted_mae,
    labels={"x": "Series", "y": "Validation MAE"},
    title="Validation MAE for All Series (Sorted)",
    width=1200,
    height=400
)
fig.update_layout(xaxis_tickangle=90)
fig.write_html("../logs/all_series_mae_sorted.html", include_plotlyjs='cdn')
fig.show()

In [ ]:
# Plot MAPE for all series (validation set), sorted by MAPE
mape_vals = [
    float(summary[k]['val_mae']) / np.mean(dataset.get_unscaled_data(k)['validation'])
    if np.mean(dataset.get_unscaled_data(k)['validation']) != 0 else np.nan
    for k in summary
]

sorted_indices = np.argsort(mape_vals)
sorted_mape = [mape_vals[i] for i in sorted_indices]
sorted_series = [series_names[i] for i in sorted_indices]

fig = px.bar(
    x=sorted_series,
    y=sorted_mape,
    labels={"x": "Series", "y": "Validation MAPE"},
    title="Validation MAPE for All Series (Sorted)",
    width=1200,
    height=400
)
fig.update_layout(xaxis_tickangle=90)
fig.write_html("../logs/all_series_mape_sorted.html", include_plotlyjs='cdn')
fig.show()

In [ ]:
# Unscaled predictions plot
top_flop = {**top_5, **bottom_5}

for series_key in top_flop:
    full_results_df = full_results_dict[series_key]
    naive_results_df = naive_predictions_dict[series_key]
    
    train_mask = ~pd.isna(full_results_df['train_predicted'])
    validation_mask = ~pd.isna(full_results_df['validation_predicted'])
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=full_results_df['timestamp'], 
        y=full_results_df['actual'], 
        mode='lines', 
        name='Actual consumption', 
        line=dict(color='blue', width=1)
    ))
    fig.add_trace(go.Scatter(
        x=full_results_df.loc[validation_mask, 'timestamp'], 
        y=full_results_df.loc[validation_mask, 'validation_predicted'], 
        mode='lines', 
        name='Validation predictions', 
        line=dict(color='red', width=2)
    ))
    fig.add_trace(go.Scatter(
        x=full_results_df.loc[validation_mask, 'timestamp'], 
        y=naive_results_df['validation'], 
        mode='lines', 
        name='Naive predictions', 
        line=dict(color='green', width=1, dash='dot')
    ))
    fig.update_layout(
        title=f'Electricity Consumption Forecast - Series {series_key}',
        xaxis_title='Date',
        yaxis_title='Electricity Consumption in Wh',
        width=1200, 
        height=600,
        hovermode='x unified'
    )
    fig.write_html(f"../logs/train_validation_truth_{series_key}.html", include_plotlyjs='cdn')

In [ ]:
# Results DataFrame for analysis
results_dict = {}

for series_key in top_flop:
    df = full_results_dict[series_key]
    validation_mask = ~pd.isna(df['validation_predicted'])
    
    validation_indices = df.index[validation_mask]
    validation_start = validation_indices.min()
    validation_end = validation_indices.max() + 1
    
    # Create results dataframe for each series
    results_df = pd.DataFrame({
        'timestamp': dataset.get_resampled_data()['deviceTimestamp'].iloc[validation_start:validation_end].reset_index(drop=True),
        'actual': df.loc[validation_start:validation_end-1, 'actual'].values,
        'predicted': df.loc[validation_start:validation_end-1, 'validation_predicted'].values
    })
    
    if GRANULARITY == Granularity.HOURLY:
        results_df['hour_of_day'] = results_df['timestamp'].dt.hour
        results_df['day_of_week'] = results_df['timestamp'].dt.day_name()
        results_df['month_of_year'] = results_df['timestamp'].dt.month
    elif GRANULARITY == Granularity.DAILY:
        results_df['day_of_week'] = results_df['timestamp'].dt.day_name()
        results_df['month_of_year'] = results_df['timestamp'].dt.month
    elif GRANULARITY == Granularity.MONTHLY:
        results_df['month_of_year'] = results_df['timestamp'].dt.month
    
    results_dict[series_key] = results_df

In [ ]:
metrics = [
    ("hour_of_day", list(range(24)), "Hour of Day"),
    ("day_of_week", ['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'], "Day of Week"),
    ("month_of_year", list(range(1, 13)), "Month of Year")
]

for col_name, labels, xlab in metrics:
    if col_name not in next(iter(results_dict.values())).columns:
        continue

    n = len(top_flop)
    cols = min(5, n)
    rows = 2 if n > 5 else 1
    fig, axes = plt.subplots(rows, cols, figsize=(4*cols, 3*rows), squeeze=False)

    for idx, series_key in enumerate(top_flop):
        ax = axes[idx//cols][idx%cols]
        df = results_dict[series_key]

        vals = []
        for lab in labels:
            data = df[df[col_name] == lab]
            if not data.empty:
                rmse = np.sqrt(np.mean((data["actual"] - data["predicted"])**2))
            else:
                rmse = np.nan
            vals.append(rmse)
        ax.bar(labels, vals, alpha=0.7)
        ax.set_title(f"Series {series_key}")
        ax.set_xlabel(xlab)
        ax.set_ylabel("RMSE")
        ax.tick_params(axis="x", rotation=45)

    plt.tight_layout()
    fig.savefig(f"../logs/rmse_by_{col_name}.svg", dpi=150)
    plt.close(fig)

In [ ]:

metric_comparison = {}
model_vs_naive = []

for series_key in predictions_dict:
    if series_key not in naive_summary:
        continue

    model_metrics = summary[series_key]
    naive_metrics = naive_summary[series_key]

    metric_comparison[series_key] = {
        "naive_rmse": float(naive_metrics["val_rmse"]),
        "naive_mae": float(naive_metrics["val_mae"]),
        "model_rmse": float(model_metrics["val_rmse"]),
        "model_mae": float(model_metrics["val_mae"])
    }

    model_vs_naive.append({
        "series": series_key,
        "naive_rmse": naive_metrics["val_rmse"],
        "model_rmse": model_metrics["val_rmse"],
        "improved": model_metrics["val_rmse"] < naive_metrics["val_rmse"]
    })

naive_df = pd.DataFrame.from_dict(metric_comparison, orient="index")
naive_df = naive_df.sort_values("naive_rmse")

print("Overall averages:")
mean_val = np.mean([dataset.get_unscaled_data(k)['validation'].mean() for k in predictions_dict])
print(f"Average time series value over all series: {mean_val:.2f}")
print(f" Naive RMSE: {naive_df['naive_rmse'].mean():.2f}, Model RMSE: {naive_df['model_rmse'].mean():.2f}")
print(f" Naive MAE:  {naive_df['naive_mae'].mean():.2f}, Model MAE:  {naive_df['model_mae'].mean():.2f}")

# Calculate MAPE (Mean Absolute Percentage Error)
naive_mape = (naive_df['naive_mae'].mean() / mean_val)
model_mape = (naive_df['model_mae'].mean() / mean_val)
print(f"MAPE: Naive {naive_mape:.2f}, Model {model_mape:.2f}")

val_mase = naive_df['model_mae'].mean() / naive_df['naive_mae'].mean()
print(f"Model MASE: {val_mase:.2f}")

# Scatter plot: naive vs model mae per series
fig, ax = plt.subplots(figsize=(7,6))
ax.scatter(naive_df["naive_mae"], naive_df["model_mae"], alpha=0.6)
ax.plot([naive_df["naive_mae"].min(), naive_df["naive_mae"].max()],
        [naive_df["naive_mae"].min(), naive_df["naive_mae"].max()], 'r--', label="y=x",)
x_min, x_max = naive_df["naive_mae"].min(), naive_df["naive_mae"].max()
xs = np.array([x_min, x_max])
ax.plot(xs, val_mase * xs, 'g--', label=f"y={val_mase:.2f}x")
ax.set_xlabel("Naive MAE")
ax.set_ylabel("Model MAE")
ax.set_title("Model vs Naive MAE per Series")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("../logs/naive_vs_model_mae.svg", dpi=150)
plt.show()

In [ ]:
# Series properties and detailed metrics calculation
series_props = {}
for k in predictions_dict:
    # 1. Get Full Unscaled Validation Data
    full_val = dataset.get_unscaled_data(k)["validation"]
    
    # 2. Align Ground Truth using the specific striding logic
    y_true = align_ground_truth(full_val, INPUT_SIZE, OUTPUT_SIZE)
    
    # 3. Get Preds (these are already aligned by the predict method)
    y_pred = predictions_dict[k]["validation"]
    
    # 4. Get Naive Metrics (from previous cell calculations)
    # We rely on the pre-calculated MAE/RMSE from Cell 24 to avoid re-generating Naive arrays here
    naive_mae = metric_comparison[k]["naive_mae"] if k in metric_comparison else np.nan
    naive_rmse = metric_comparison[k]["naive_rmse"] if k in metric_comparison else np.nan

    # Properties calculation (on the full original series for context)
    s = pd.Series(full_val.flatten())
    mean = s.mean()
    std = s.std()
    coeff_var = std / mean if mean != 0 else np.nan
    median = s.median()
    mean_abs = np.mean(np.abs(s))
    maxv, minv = s.max(), s.min()
    skewness = s.skew()
    kurt = s.kurtosis()
    pct_zero = (s == 0).mean() if len(s) > 0 else np.nan
    
    try:
        if GRANULARITY == Granularity.HOURLY:
            autocorr = s.autocorr(lag=24)
        elif GRANULARITY == Granularity.DAILY:
            autocorr = s.autocorr(lag=7)
        else:
            autocorr = np.nan
    except Exception:
        autocorr = np.nan

    # Compute Detailed KPIs (R2, Bias, MAPE) on the ALIGNED windows
    # Note: y_true and y_pred are now flattened arrays of valid windows
    if len(y_true) == len(y_pred) and len(y_true) > 0:
        model_mae = float(mean_absolute_error(y_true, y_pred))
        model_rmse = float(root_mean_squared_error(y_true, y_pred))
        
        # Mask zeros for MAPE
        with np.errstate(divide='ignore', invalid='ignore'):
            mape_arr = np.abs((y_true - y_pred) / y_true)
            mape_arr = mape_arr[np.isfinite(mape_arr)]
        model_mape = float(np.mean(mape_arr)) * 100 if len(mape_arr) > 0 else np.nan
        
        model_r2 = float(r2_score(y_true, y_pred)) if len(y_true) > 1 else np.nan
        bias = float(np.mean(y_pred - y_true))
    else:
        model_mae = model_rmse = model_mape = model_r2 = bias = np.nan

    # Per-series MASE
    per_mase = model_mae / naive_mae if (naive_mae and not np.isnan(naive_mae) and naive_mae != 0) else np.nan
    mae_reduction_pct = (naive_mae - model_mae) / naive_mae * 100 if (naive_mae and naive_mae != 0 and not np.isnan(naive_mae)) else np.nan

    series_props[k] = {
        "mean": mean,
        "std": std,
        "cv": coeff_var,
        "median": median,
        "mean_abs": mean_abs,
        "max": maxv,
        "min": minv,
        "skew": skewness,
        "kurtosis": kurt,
        "pct_zero": pct_zero,
        "autocorr": autocorr,
        "model_mae": model_mae,
        "model_rmse": model_rmse,
        "model_mape_pct": model_mape,
        "model_r2": model_r2,
        "model_bias": bias,
        "naive_mae": naive_mae,
        "naive_rmse": naive_rmse,
        "per_mase": per_mase,
        "mae_reduction_pct": mae_reduction_pct,
        "n_val": len(y_true)
    }

df_stats = pd.DataFrame.from_dict(series_props, orient="index")

# Save CSV
os.makedirs("../logs", exist_ok=True)
df_stats.to_csv("../logs/series_stats.csv")

# ... (Rest of the plotting code in this cell remains the same) ...
# Plots: scatter property vs model/naive MAE, and improvement
plot_cols = [("mean", "Mean"), ("std", "Std Dev"), ("cv", "Coeff of Var"), ("mean_abs", "Mean Abs (magnitude)"), ("autocorr", "Seasonal autocorr")]
for col, label in plot_cols:
    plt.figure(figsize=(6,4))
    sns.scatterplot(data=df_stats, x=col, y="model_mae", label="Model MAE", alpha=0.7)
    sns.scatterplot(data=df_stats, x=col, y="naive_mae", label="Naive MAE", alpha=0.6)
    plt.xlabel(label); plt.ylabel("MAE"); plt.title(f"{label} vs MAE (model vs naive)")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"../logs/{col}_vs_mae.png", dpi=150)
    plt.close()

# Plot improvement (percentage) vs properties
plt.figure(figsize=(7,5))
sns.scatterplot(data=df_stats, x="mean_abs", y="mae_reduction_pct", hue="model_r2", palette="viridis", alpha=0.8)
plt.axhline(0, color="r", linestyle="--", label="no improvement")
plt.xlabel("Mean Abs (magnitude)"); plt.ylabel("MAE reduction vs naive (%)")
plt.title("MAE reduction (%) vs Series magnitude")
plt.legend()
plt.tight_layout()
plt.savefig("../logs/mae_reduction_vs_magnitude.png", dpi=150)
plt.close()

# Distribution of per-series MASE and MAE reduction
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
sns.histplot(df_stats["per_mase"].dropna(), bins=30, kde=False)
plt.title("Per-series MASE (model / naive)")
plt.xlabel("MASE")
plt.subplot(1,2,2)
sns.histplot(df_stats["mae_reduction_pct"].dropna(), bins=30, kde=False)
plt.title("MAE reduction (%) (naive -> model)")
plt.xlabel("% reduction")
plt.tight_layout()
plt.savefig("../logs/mase_and_reduction_hist.png", dpi=150)
plt.close()

# Correlation heatmap between properties and KPIs
corr_cols = ["mean","std","cv","mean_abs","autocorr","model_mae","model_rmse","model_mape_pct","model_r2","model_bias","naive_mae","per_mase","mae_reduction_pct"]
corr = df_stats[corr_cols].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt=".2f", cmap="vlag", center=0)
plt.title("Correlation between series properties and KPIs")
plt.tight_layout()
plt.savefig("../logs/props_kpis_correlation.svg", dpi=150)
plt.close()

# Summary counts: how many series improved vs worsened
improved = (df_stats["mae_reduction_pct"] > 0).sum()
worsened = (df_stats["mae_reduction_pct"] <= 0).sum()
print(f"Series improved vs naive (by MAE): {improved} improved / {len(df_stats)} total ({improved/len(df_stats):.2%})")

In [ ]:
# Logging with MLFlow
if USE_MLFLOW:
    mlflow.set_tracking_uri("file:../logs/mlruns")
    try:
        mlflow.create_experiment("Energy_Consumption_Forecast")
    except:
        mlflow.set_experiment("Energy_Consumption_Forecast")

    with mlflow.start_run():
        # Model Architecture
        if SAVE_MODEL:
            mlflow.pytorch.log_model(model, name="LSTM forecaster")
        
        mlflow.log_artifact("../logs/model_architecture.txt")
        mlflow.log_param("granularity", GRANULARITY.value)
        mlflow.log_param("input_size", INPUT_SIZE)
        mlflow.log_param("output_size", OUTPUT_SIZE)
        mlflow.log_param("learning_rate", LR)
        mlflow.log_param("loss_function", loss_fn._get_name())
        mlflow.log_param("scaler", SCALER.__name__)
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("n_epochs", N_EPOCHS)
        mlflow.log_param("model_type", ModelClass.__name__)
        mlflow.log_param("N_SERIES", N_SERIES)
        mlflow.log_param("USE_TIME_COVARIATES", USE_TIME_COVARIATES)

        # Model training results
        mlflow.log_metric("train_rmse", history["train_rmse"][-1])
        mlflow.log_metric("train_mae", history["train_mae"][-1])
        mlflow.log_metric("train_mae_avg", np.mean([m['train_mae'] for m in summary.values() if not np.isnan(m['train_mae'])]))
        mlflow.log_metric("val_mase", val_mase)
        v1 = summary.get("value_1", {})
        v1_train_mae = v1.get("train_mae", np.nan)
        v1_val_mae = v1.get("val_mae", np.nan)
        if not np.isnan(v1_train_mae):
            mlflow.log_metric("value_1_train_mae", float(v1_train_mae))
        if not np.isnan(v1_val_mae):
            mlflow.log_metric("value_1_val_mae", float(v1_val_mae))
        mlflow.log_metric("val_rmse", history["val_rmse"][-1])
        mlflow.log_metric("val_mae", history["val_mae"][-1])
        mlflow.log_metric("val_mae_avg", np.mean([m['val_mae'] for m in summary.values() if not np.isnan(m['val_mae'])]))
        mlflow.log_metric("total_training_time_sec", total_training_time)
        mlflow.log_artifact("../logs/rmse.png")
        mlflow.log_artifact("../logs/epoch_times.png")
        mlflow.log_artifact("../logs/attention_heatmap.html")

        # Log all series prediction results
        if 'hour_of_day' in next(iter(results_dict.values())).columns:
            mlflow.log_artifact(f"../logs/rmse_by_hour_of_day.png")
        if 'day_of_week' in next(iter(results_dict.values())).columns:
            mlflow.log_artifact(f"../logs/rmse_by_day_of_week.png")
        if 'month_of_year' in next(iter(results_dict.values())).columns:
            mlflow.log_artifact(f"../logs/rmse_by_month_of_year.png")
        
        for series_key in top_flop:
            mlflow.log_artifact(f"../logs/train_validation_truth_{series_key}.html")

        mlflow.log_artifact("../logs/all_series_mae_sorted.html")
        mlflow.log_artifact("../logs/all_series_mape_sorted.html")
        mlflow.log_artifact("../logs/naive_vs_model_mae.png")

        mlflow.log_artifact("../logs/series_stats.csv")
        for col, _ in plot_cols:
            mlflow.log_artifact(f"../logs/{col}_vs_mae.png")
        mlflow.log_artifact("../logs/mae_reduction_vs_magnitude.png")
        mlflow.log_artifact("../logs/mase_and_reduction_hist.png")
        mlflow.log_artifact("../logs/props_kpis_correlation.png")
        